In [0]:
%pip install --upgrade --force-reinstall /dbfs/tmp/insurance-0.0.1-py3-none-any.whl

In [0]:
%restart_python

In [0]:
import os
import time
from typing import Dict, List
from loguru import logger
import requests
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession

from insurance.config import ProjectConfig
from insurance.serving.model_serving import ModelServing


In [0]:

spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

In [0]:
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
catalog_name = config.catalog_name
schema_name = config.schema_name

In [0]:
# Initialize feature store manager
model_serving = ModelServing(
    model_name=f"{catalog_name}.{schema_name}.insurance_model_basic", endpoint_name="insurance-model-serving-basic"
)

In [0]:
# Deploy the model serving endpoint
model_serving.deploy_or_update_serving_endpoint()

In [0]:
# Create a sample request body
required_columns = ["age", "bmi", "children", "sex", "smoker", "region"]


In [0]:
# Sample 1000 records from the training set
test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").toPandas()
test_set = test_set.drop(columns=["Id"])

sampled_records = test_set[required_columns].sample(n=100, replace=True).to_dict(orient="records")
dataframe_records = [[record] for record in sampled_records]

logger.info(test_set.dtypes)
logger.info(dataframe_records[0])


In [0]:
import json
print(json.dumps(dataframe_records[0][0], indent=2))

In [0]:
def call_endpoint(record):
    """
    Calls the model serving endpoint with a given input record.
    """
    serving_endpoint = f"https://{os.environ['DBR_HOST']}/serving-endpoints/insurance-model-serving-basic/invocations"

    response = requests.post(
        serving_endpoint,
        headers={"Authorization": f"Bearer {os.environ['DBR_TOKEN']}"},
        json={"dataframe_records": record},
    )
    return response.status_code, response.text

In [0]:
status_code, response_text = call_endpoint(dataframe_records[0])
print(f"Response Status: {status_code}")
print(f"Response Text: {response_text}")

In [0]:
# Load test
for i in range(len(dataframe_records)):
    status_code, response_text = call_endpoint(dataframe_records[i])
    print(f"Response Status: {status_code}")
    print(f"Response Text: {response_text}")
    time.sleep(0.2)